In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


# Pickle Load


In [3]:
# Import pickle Package
import pickle

pkl_filename = "Resources/pickle/Pickle_45_Model.pkl"  
# Load the Model back from file
with open(pkl_filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)
Pickled_LR_Model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [4]:

pkl_filename2 = "Resources/pickle/pickle_45_Model2.pkl"  
# Load the Model back from file
with open(pkl_filename2, 'rb') as file2:  
    Pickled_Model2 = pickle.load(file2)
Pickled_Model2


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# API Call 


In [5]:
#set up API authonatication
client_id = "5028c957611248149d8c04007258f254"
client_secret = "aa56a507cb944f488f8835062a94115c"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
#create link to input button for Artist and Track
Artist="Lionel Richie"
Track="Tonight Will Be Alright"


In [7]:
features = ['0', 'Artist', 'Track', 'Unnamed: 0', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'followers', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo',
       'time_signature', 'valence']
features = pd.DataFrame(columns = features)
features.head()

,0,Artist,Track,Unnamed: 0,acousticness,danceability,duration_ms,energy,followers,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,valence


In [8]:
try: 
        artistname= Artist
        search_str =  Track
        print(artistname+ ","+ search_str )
        result= sp.search(search_str+":"+artistname,type="track,artist")
        print(result["tracks"]["items"][0]["id"])
        
        track_id = result["tracks"]["items"][0]["id"]
        artist_id = result['tracks']['items'][0]["album"]['artists'][0]['id']
        
        features_i = sp.audio_features(track_id)
        features_i = pd.DataFrame(features_i)
        
        #features["genres"] = sp.artist(artist_id)['genres']
        features_i["followers"] = sp.artist(artist_id)['followers']['total']
        features_i["tempo"] = sp.audio_analysis(track_id)['track']['tempo']
        features_i["popularity"] = sp.track(track_id)["popularity"]
        features_i["Artist"] = artistname
        features_i["Track"] = search_str
        features = features.append(features_i)
        
except: 
        print ('data not found')

Lionel Richie,Tonight Will Be Alright
2Jr12fpsMdIM6weoUDMf9u


In [9]:
features = features[['Artist', 'Track', 'followers','instrumentalness','popularity','acousticness', 'danceability', 'duration_ms', 'energy', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']]

In [10]:
features


,Artist,Track,followers,instrumentalness,popularity,acousticness,danceability,duration_ms,energy,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Lionel Richie,Tonight Will Be Alright,1647784,0.0141,39,0.0591,0.646,307413,0.84,8,0.0516,-8.213,1,0.0283,125.188,4,0.716


In [11]:
features_x = features[['followers','instrumentalness','acousticness', 'danceability', 'duration_ms', 'energy', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']]

In [12]:
print(features_x.shape)

(1, 14)


# API Predict A - Linear Regression 


In [13]:
# Predict the Labels using the reloaded Model
Y_predict2 = Pickled_LR_Model.predict(features_x)  
Y_predict2

array([[43.27381286]])

In [14]:
features_y = features[['popularity']]
print(features_y['popularity'])


0    39
Name: popularity, dtype: object


In [15]:
print(f"Spotify song poularity: {features['popularity']}") 
print(f"Model predicted popularity: {Y_predict2}")       

Spotify song poularity: 0    39
Name: popularity, dtype: object
Model predicted popularity: [[43.27381286]]


# API Predict B - Random Forest  


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [17]:
#song_prediction_popular(1) or not_popular(0)
Y_predict3 = Pickled_Model2.predict(features_x)  
Y_predict3

array([0], dtype=int64)

# Pickle Accuracy Confirmations  - Only for checking 

In [18]:
# Model 2 - pickle accuracy check 
file4 = "Resources/test_csvs/X_valid.csv"
X_valid = pd.read_csv(file4)
file5 = "Resources/test_csvs/y_valid.csv"
y_valid = pd.read_csv(file5)

In [19]:
RFC_Model2 = Pickled_Model2
#RFC_Model2.fit(X_train, y_train)
RFC_Predict2 = RFC_Model2.predict(X_valid)
RFC_Accuracy2 = accuracy_score(y_valid, RFC_Predict2)
print("Accuracy: " + str(RFC_Accuracy2))
RFC_AUC2 = roc_auc_score(y_valid, RFC_Predict2) 
print("AUC: " + str(RFC_AUC2))

Accuracy: 0.9186271796291171
AUC: 0.8841972844239776


In [20]:
# Model 1 - pickle accuracy check
file6 = "Resources/test_csvs/X_test.csv"
X_test = pd.read_csv(file6)
file7 = "Resources/test_csvs/y_test.csv"
y_test = pd.read_csv(file7)

In [21]:
# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values
# Calculate the Score 
score = Pickled_LR_Model.score(X_test, y_test)  
# Print the Score
print("Test score: {0:.2f} ".format(score))  
# Predict the Labels using the reloaded Model
Y_predict = Pickled_LR_Model.predict(X_test)  
Y_predict

Test score: 0.22 


array([[40.86042781],
       [36.20730164],
       [44.66136192],
       ...,
       [24.99936195],
       [53.1435068 ],
       [37.83834929]])